In [1]:
import csv
import json
import sqlite3
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime
import torch
import pickle
import random
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import os
from torchvision import models
from torch import nn as nn
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn import svm
from copy import deepcopy


/home/liuxiao/anaconda3/envs/torchenv/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [14]:
def plot_learning_curve_single_subject(marks_conf,counterbalance):

    fig, ax = plt.subplots(figsize=(5, 3))
    rect = patches.Rectangle((0,-0.2), 3, 1.4, linewidth=1,  facecolor='yellow', alpha=0.5)#edgecolor='r',
    conf_mat = marks_conf[9]
    marks_acc = [(conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()]
    for i in range(1,6):
        conf_mat = marks_conf[(i+1)*8-1] - marks_conf[i*8-1]
        acc = (conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()
        marks_acc.append(acc)

    ax.scatter(range(len(marks_acc)),marks_acc,c='black',s=60)

    for i in range(len(marks_acc)-1):
        ax.plot([i,i+1],[marks_acc[i],marks_acc[i+1]],c='black')
    plt.yticks([_*0.1 for _ in range(0,11,2)],[round(_*0.1,1) for _ in range(0,11,2)],fontsize=20)
    plt.xticks(range(len(marks_acc)),[_*8 for _ in range(1,len(marks_acc)+1)],fontsize=20)
    plt.suptitle('counterbalance: {}'.format(counterbalance), fontsize=20)
    # Add the patch to the Axes
    ax.add_patch(rect)
    ax.xaxis.set_ticks_position('none') 
    ax.yaxis.set_ticks_position('none') 
    ax.spines[['right', 'left']].set_visible(False)

def construct_confmat():
    labels = ['h_azure','h_detectron','h_google','h_rekognition']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices


def print_metadata(data,dummy_mark='NA'):
    print("====="*20)
    datetime_object1 = datetime.strptime(data[11].split('.')[0], '%Y-%m-%d %H:%M:%S')
    datetime_object2 = datetime.strptime(data[13].split('.')[0], '%Y-%m-%d %H:%M:%S')
    time_used = datetime_object2 - datetime_object1
    time_used = round(time_used.seconds/60,1)
    print("[Finished Date]",datetime_object2)
    print("[Time Used] {} mins".format(time_used))
    print("[Questionnaire]", json.loads(data[-1])["questiondata"])
    print("[Counterbalance]", json.loads(data[-1])["counterbalance"])
    print("[Dummy Test Acc]", dummy_mark)
    return time_used, json.loads(data[-1])["counterbalance"]

def get_gt_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'a':'Human','b':'Machine','c':'Machine','d':'Machine','e':'Machine','f':'Catch'}[pagename.split('_')[1]]

def get_model_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'a':'','b':'azure','c':'detectron','d':'google','e':'rekognition'}[pagename.split('_')[1]]

def get_imbedding_from_pagename(pagename,d):
    model = get_model_from_pagename(pagename)
    key = ('{}.jpg'.format(pagename.split('_')[0]),model)
    x = d[key][1]
    y = {'Human':0,'Machine':1}[get_gt_from_pagename(pagename)]
    return x,y
    
with open("/home/liuxiao/TuringGithub/XiaoData/objectdetection_openai_embedding_dict.pkl",'rb') as f:
    objectdetection_openai_embedding_dict = pickle.load(f)


conn = sqlite3.connect("ec2_db/turing_feedback_obj.db")
cursor = conn.cursor()
cursor.execute("select * from tablechairDB;")

label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'azure':1,'detectron':1,'google':1,'catch':1,'rekognition':1}
oneWord_curr_d = {'0':True,'1':False} # dummy test gt
topic_d = {'1':True,'2':False} # user response to dummy tests
modelname_dict = {'catch':'CatchTrial','azure':'Azure','detectron':'Detectron','rekognition':'Rekognition','google':'Google'}




count = 0
participant_count = 0
trials_dict = {}
all_data = cursor.fetchall()
completed_counterbalances = []


confmat_overall = construct_confmat()
catch_trial_mat_overall = np.zeros((1,2))
swap_trial_mat_overall = np.zeros((1,2))
confmat_overall_AI = construct_confmat()

ai_judge = {}
trialnum = 0
for i in range(len(all_data)):
    if all_data[i][15] in [4]:
#         print(all_data[i][11])
        data_dict = json.loads(all_data[i][17])
        if data_dict['mode'] != 'live':
            continue
        # filter out subjects using dummy trial accuracy
        # original threshold in the paper: 7/12
        dummy_mark = 0
        nTrials = 100
        threshold = nTrials*(7/12) # change to number of total trials 
        for dat in data_dict['data']:

            if dat['trialdata']['phase'] == 'TEST':
                present = dat['trialdata']['oneWord_curr'] == '0'
                response_present = dat['trialdata']['topic'] == '1'
                if present == response_present:
                    dummy_mark += 1
        if dummy_mark < threshold:
            continue
        
        
        # construct conf matrices
        time_used, counterbalance = print_metadata(all_data[i],"{}/{}".format(dummy_mark,nTrials))
        completed_counterbalances.append(counterbalance)
#         marks = []
#         marks_conf = []
        confmatrices = construct_confmat() # without catch
        confmat_AI = construct_confmat()
        catch_trial_mat = np.zeros((1,2))
        swap_trial_mat = np.zeros((1,2))
        marks = []
        marks_conf = []
        participant_count += 1
        nTrial = 0
        
        X_train, y_train, X_test, y_test = [], [], [], []
        test_samples = []
        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':
                
                nTrial+=1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                trial_data['workerData'] = data_dict['questiondata']
                trial_data['trialData'] = dat['trialdata']

                
#                 trials_dict['Trial_'+str(count)] = trial_data
#                 trial_data = trial_data['trialData']
                trial_data['trialData']['correct response'] = get_gt_from_pagename(trial_data['trialData']['page_name'])
                trial_data['trialData']['choose response'] = trial_data['trialData']['Aclass']
                
#                 print(trial_data)
                if trial_data['trialData']['correct response'] == 'Catch':
                    catch_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    catch_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif trial_data['trialData']['correct response'] == 'Swapped':
                    swap_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    swap_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif nTrial > 40:  
                    confmatrices['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    confmat_overall['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    if trial_data['trialData']['correct response'] == "Machine":
                        model = get_model_from_pagename(trial_data['trialData']['page_name'])
                        confmatrices['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                        confmat_overall['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                    marks.append(1 if trial_data['trialData']['correct response'] == trial_data['trialData']['choose response'] else 0)
                    marks_conf.append(deepcopy(confmatrices['h_machine']))
                    
                    ### === Add for AI === ### 
                    x,y = get_imbedding_from_pagename(trial_data['trialData']['page_name'],objectdetection_openai_embedding_dict)   
                    X_test.append(x)
                    y_test.append(y)
                    trial_data['trialData']["machine_groundtruth"] = get_model_from_pagename(trial_data['trialData']['page_name'])
                    test_samples.append(deepcopy(trial_data['trialData']))
                    
                elif nTrial <= 40:
                    ### === Add for AI === ###
                    x,y = get_imbedding_from_pagename(trial_data['trialData']['page_name'],objectdetection_openai_embedding_dict)   
                    X_train.append(x)
                    y_train.append(y)
                    
#                 trial_data['trialData'] = dat['trialdata']
                if trial_data['trialData']['correct response']  in ['Catch','Swapped'] or nTrial <=40:
                    continue
                
                count += 1
        print('Human Judge',confmatrices)
        print('catch',catch_trial_mat)                
        ### ===================== AI judge ======================== ###
#         assert len(X_train) == 40 and len(y_train) == 40 and len(X_test) == 40 and len(y_test) == 40
        clf = svm.SVC(verbose=True)
        clf.fit(list(X_train), y_train)
        predictions = clf.predict(X_test)
        pred_indx = 0
                
        for sample in test_samples:
            
            trialnum+=1
            response_speaker = ''
            if(predictions[pred_indx]==0):
                response_speaker = "human"
            elif(predictions[pred_indx]==1):
                response_speaker = "machine"

            confmat_AI['h_machine'][label_dict[sample['correct response']]][predictions[pred_indx]] += 1
            confmat_overall_AI['h_machine'][label_dict[sample['correct response']]][predictions[pred_indx]] += 1

            if sample['correct response'].lower() == 'machine':  
                model = sample["machine_groundtruth"]
                confmat_AI['h_{}'.format(model)][0][predictions[pred_indx]] += 1
                confmat_overall_AI['h_{}'.format(model)][0][predictions[pred_indx]] += 1

            trialdata = {"response_speaker": response_speaker, "groundtruth":sample["correct response"].lower(), "machine_groundtruth":sample["machine_groundtruth"]}
            ai_judge['Trial_Obj_'+str(trialnum)] = {"trialData": trialdata}
            pred_indx+=1  

#         plot_learning_curve_single_subject(marks_conf,participant_count)
        print('AI Judge',confmat_AI)

#         print('swap',swap_trial_mat)
#     else:
#         print(all_data[i][15])
conn.close()

[Finished Date] 2025-04-02 10:44:07
[Time Used] 18.2 mins
[Questionnaire] {'country': 'United States', 'age': '51', 'engagement': '6', 'difficulty': '6', 'mode': 'live', 'gender': 'male', 'education': 'bachelor', 'ai-experience': '2', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 85/100
Human Judge {'h_machine': array([[17.,  3.],
       [ 6., 14.]]), 'h_azure': array([[2., 3.]]), 'h_detectron': array([[1., 4.]]), 'h_google': array([[2., 3.]]), 'h_rekognition': array([[1., 4.]])}
catch [[ 0. 10.]]
[LibSVM]*.*
optimization finished, #iter = 40
obj = -21.109168, rho = 0.107530
nSV = 39, nBSV = 24
Total nSV = 39
AI Judge {'h_machine': array([[16.,  4.],
       [10., 10.]]), 'h_azure': array([[2., 3.]]), 'h_detectron': array([[1., 4.]]), 'h_google': array([[4., 1.]]), 'h_rekognition': array([[3., 2.]])}
[Finished Date] 2025-04-02 10:44:52
[Time Used] 17.5 mins
[Questionnaire] {'country': 'United States', 'age': '51', 'engagement': '9', 'difficulty': '9', 'mode': 'liv

[LibSVM]*.*
optimization finished, #iter = 40
obj = -22.272982, rho = 0.111741
nSV = 40, nBSV = 25
Total nSV = 40
AI Judge {'h_machine': array([[17.,  3.],
       [ 5., 15.]]), 'h_azure': array([[1., 4.]]), 'h_detectron': array([[1., 4.]]), 'h_google': array([[1., 4.]]), 'h_rekognition': array([[2., 3.]])}
[Finished Date] 2025-04-02 11:35:00
[Time Used] 16.5 mins
[Questionnaire] {'country': 'United States', 'age': '45-50', 'engagement': '9', 'difficulty': '8', 'mode': 'live', 'gender': 'male', 'education': 'bachelor', 'ai-experience': '3', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 82/100
Human Judge {'h_machine': array([[15.,  5.],
       [ 9., 11.]]), 'h_azure': array([[0., 5.]]), 'h_detectron': array([[4., 1.]]), 'h_google': array([[2., 3.]]), 'h_rekognition': array([[3., 2.]])}
catch [[ 0. 10.]]
[LibSVM]*.*
optimization finished, #iter = 44
obj = -20.793222, rho = -0.008785
nSV = 40, nBSV = 23
Total nSV = 40
AI Judge {'h_machine': array([[14.,  6.],
      

obj = -21.294436, rho = 0.150432
nSV = 40, nBSV = 24
Total nSV = 40
AI Judge {'h_machine': array([[20.,  0.],
       [ 2., 18.]]), 'h_azure': array([[0., 5.]]), 'h_detectron': array([[0., 5.]]), 'h_google': array([[1., 4.]]), 'h_rekognition': array([[1., 4.]])}
[Finished Date] 2025-04-02 12:53:41
[Time Used] 29.7 mins
[Questionnaire] {'country': 'United States', 'age': '36-40', 'engagement': '10', 'difficulty': '5', 'mode': 'live', 'gender': 'female', 'education': 'high school', 'ai-experience': '0', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 82/100
Human Judge {'h_machine': array([[14.,  6.],
       [ 8., 12.]]), 'h_azure': array([[1., 4.]]), 'h_detectron': array([[4., 1.]]), 'h_google': array([[1., 4.]]), 'h_rekognition': array([[2., 3.]])}
catch [[1. 9.]]
[LibSVM]*.*
optimization finished, #iter = 41
obj = -22.871612, rho = -0.003271
nSV = 40, nBSV = 25
Total nSV = 40
AI Judge {'h_machine': array([[19.,  1.],
       [ 4., 16.]]), 'h_azure': array([[1., 4.]]

[Finished Date] 2025-04-02 14:04:33
[Time Used] 19.1 mins
[Questionnaire] {'country': 'United States', 'age': '36-40', 'engagement': '5', 'difficulty': '3', 'mode': 'live', 'gender': 'male', 'education': 'bachelor', 'ai-experience': '3', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 84/100
Human Judge {'h_machine': array([[14.,  6.],
       [11.,  9.]]), 'h_azure': array([[4., 1.]]), 'h_detectron': array([[3., 2.]]), 'h_google': array([[1., 4.]]), 'h_rekognition': array([[3., 2.]])}
catch [[1. 9.]]
[LibSVM]*.*
optimization finished, #iter = 41
obj = -21.618208, rho = 0.158329
nSV = 40, nBSV = 25
Total nSV = 40
AI Judge {'h_machine': array([[17.,  3.],
       [ 7., 13.]]), 'h_azure': array([[2., 3.]]), 'h_detectron': array([[4., 1.]]), 'h_google': array([[1., 4.]]), 'h_rekognition': array([[0., 5.]])}
[Finished Date] 2025-04-02 14:07:07
[Time Used] 17.7 mins
[Questionnaire] {'country': 'United States', 'age': '51', 'engagement': '10', 'difficulty': '4', 'mode': 'l

In [56]:
with open("/home/liuxiao/TuringGithubRunnable/dominant_color_recognition/Plot/Data_all/humanjudge_Color_Feedback_testphase.json", 'w') as f:
    json.dump(trials_dict,f)

In [18]:
json.dump(ai_judge, open("/home/liuxiao/TuringGithubRunnable/multi_label_prediction/Plot/Data_all/ai_judge_feedback_object.json","w"))
json.dump(ai_judge, open("/home/liuxiao/TuringGithub/XiaoData/all_results/ai_judge_feedback_object.json","w"))


Bad pipe message: %s [b"\xbedl\xf8'\x7fr\x87\xa2\xaf\xdaZ=~\xf1\xe4\x1af\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a"]
Bad pipe message: %s [b'\x86\xa4r\xc5]\x8a>\xf5\xc1\xf5|\xd49\xc4\xff@[\xd7\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b']
Bad pipe message: %s [b"\xfe\xdd\xe0\x84G$\x83\x14 :'\xae\\\xb4l\xcc\xde\x94\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12"]
Bad pipe message: %s [b'|\xfa\xb1h7\x90C\xb5`\xa0\\\xd88t\xedp\x8f_\x00\x02\xbc\x00\x00\x00\x01\x00']
Bad pipe message: %s [b'\x03']
Bad pipe message: %s [b'\xbd\xbb\xe4\x83\xeaQM\xfc\x14n\x8a\x9bd~\xb4\x8a\xd7\xbb\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x

In [12]:
confmat_overall_AI

{'h_machine': array([[840., 160.],
        [295., 705.]]),
 'h_azure': array([[ 47., 203.]]),
 'h_detectron': array([[ 93., 157.]]),
 'h_google': array([[105., 145.]]),
 'h_rekognition': array([[ 50., 200.]])}

In [40]:
856/(856+628),686/(686.+ 489),231/(231.+ 170),217/(217.+148)

(0.5768194070080862,
 0.5838297872340426,
 0.5760598503740648,
 0.5945205479452055)

In [52]:
confmat_overall

{'h_machine': array([[867., 617.],
        [505., 670.]]),
 'h_azure': array([[172., 237.]]),
 'h_histogram': array([[154., 247.]]),
 'h_google': array([[179., 186.]])}

In [16]:
len(ai_judge)

2000